In [3]:
import cv2
import pandas as pd
import numpy as np
df_1 = pd.read_csv("data/Slideseq_1_glomeruli_position.csv", header=0, index_col=0)
df_2 = pd.read_csv("data/Slideseq_2_glomeruli_position.csv", header=0, index_col=0)
df_1["AP"] = df_1["AP"] / 4000
df_2["AP"] = df_2["AP"] / 4000
df_1.sort_values("OR", inplace=True)
df_2.sort_values("OR", inplace=True)
print(df_1.head())
print(df_2.head())
ORs_1 = df_1.OR.unique()
ORs_2 = df_2.OR.unique()
shared_ORs = [x for x in ORs_1 if x in ORs_2]
coords_1 = df_1[df_1["OR"].isin(shared_ORs)][["AP", "DV"]].values
coords_2 = df_2[df_2["OR"].isin(shared_ORs)][["AP", "DV"]].values
H, _ = cv2.findHomography(coords_1, coords_2)
n = coords_2.shape[0]
new_coords_1 = np.zeros((n, 3))
for i in range(n):
    pt1 = np.array([coords_1[i,0], coords_1[i,1], 1])
    pt1 = pt1.reshape(3, 1)
    pt2 = np.dot(H, pt1)
    pt2 = pt2/pt2[2]
    new_coords_1[i,0] = pt2[0]
    new_coords_1[i,1] = pt2[1]
with open("data/transformed_coords_1to2.txt", "w") as f:
    for index, row in df_2.iterrows():
        print(row["OR"], row["AP"], row["DV"], 2, file=f, sep="\t")
    for index, row in df_1.iterrows():
        pt1 = np.array([row["AP"], row["DV"], 1])
        pt1 = pt1.reshape(3, 1)
        pt2 = np.dot(H, pt1)
        pt2 = pt2/pt2[2]
        print(row["OR"], pt2[0,0], pt2[1,0], 1, file=f, sep="\t")

     X        OR  slide           ML           DV    AP
18  31  Olfr1000      7  -771.154506 -1973.983747   7.0
33  56  Olfr1018     10  -718.368031 -2787.781019  10.0
24  46  Olfr1030      9  -653.009496 -1383.292778   9.0
25  47  Olfr1031      9  -710.608633 -1251.281594   9.0
34  57  Olfr1045     10 -1295.912350 -1593.913807  10.0
     X        OR  slide           ML           DV    AP
19  32  Olfr1000     11 -1674.094228 -1967.999543  11.0
44  73  Olfr1013     17 -1149.473539 -1449.529952  17.0
11  14  Olfr1030      9 -1015.996164 -1303.241413   9.0
20  33  Olfr1048     11  -807.635396 -1108.751969  11.0
25  40  Olfr1086     12 -1155.866224  -311.059204  12.0
